This notebook is for getting the result from pre-train model

In [ ]:
import torch
import torchvision
import h5py
import cv2 as cv
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from PIL import Image
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

In [ ]:
device = torch.device('cuda')

In [ ]:
weights = torchvision.models.detection.FCOS_ResNet50_FPN_Weights.DEFAULT
model = torchvision.models.detection.fcos_resnet50_fpn(weights=weights).to(device)
model.eval();
transform = weights.transforms()

In [ ]:
path = Path('/home/tam/') / 'git' / 'cvppp2017_dataset'
training = h5py.File(path / 'CVPPP2017_training_images.h5')
truth = h5py.File(path / 'CVPPP2017_training_truth.h5')

In [ ]:
dataset = 'A4' # Change data set from A1, A2, A3 or A4

In [ ]:
result = {}
with torch.no_grad():
    for plant_id in tqdm(training[dataset].keys()):
        image = Image.fromarray(training[dataset][plant_id]['rgb'][:, :, :3])
        image = transform(image).to(device)
        prediction = model([image])
        image.detach()
        result.update({plant_id: prediction})

In [ ]:
for plant_id in tqdm(training[dataset].keys()):
    image = Image.fromarray(training[dataset][plant_id]['rgb'][:, :, :3])
    box = draw_bounding_boxes(
        torchvision.transforms.Compose([torchvision.transforms.PILToTensor()])(image), 
        result[plant_id][0]["boxes"],
        [weights.meta['categories'][i] for i in result[plant_id][0]["labels"]],
        colors='red')
    to_pil_image(box.detach()).save(path / 'result' / f'{dataset}-{plant_id}-pretrain.png')